# <font color=#257C53 size=6> Datasets do Programa Universidade para Todos (ProUni)</font>
# <font color=#999999 size=5> Processo de ETL para um banco de dados no SQL Server </font>
***

# <font color=#257C53 size=5> 1 CONHECENDO OS DADOS </font>
***

O conjunto de datasets trabalhado neste relatório é referente as bolsas concedidas e perfil dos beneficiários do <b>Programa Universidade para Todos</b>, obtidos no Portal Brasileiro de Dados Abertos (https://dados.gov.br/dataset/mec-prouni). Ao todo, até data de desenvolvimento deste relatório, foram disponibilizados dezesseis arquivos <i>.csv</i> referentes aos anos de <b>2005 a 2020</b>.

Embora a identificação dos arquivos <i>.csv</i> tenha sido alterado para fins de desenvolvimento de projeto, nenhum dataset sofreu nenhuma interferência externa às etapas deste relatório.
***

###  Sobre o programa
O Programa Universidade para Todos - Prouni tem como finalidade a concessão de bolsas de estudo integrais e parciais em cursos de graduação e sequenciais de formação específica, em instituições de ensino superior privadas que oferece, em contrapartida, isenção de tributos àquelas instituições que aderem ao Programa.

Dirigido aos estudantes egressos do ensino médio da rede pública ou da rede particular na condição de bolsistas integrais, com renda familiar per capita máxima de três salários mínimos, o Prouni conta com um sistema de seleção informatizado e impessoal, que confere transparência e segurança ao processo. Os candidatos são selecionados pelas notas obtidas no Exame Nacional do Ensino Médio - Enem conjugando-se, desse modo, inclusão à qualidade e mérito dos estudantes com melhores desempenhos acadêmicos.
***

# <font color=#257C53 size=5> 2 MODELAGEM DE DADOS </font>
***

Os dados tratados serão formatados para um banco de dados SQL com a seguinte modelagem.
<img src="Diagrama geral.png">

# <font color=#257C53 size=5> 3 PROCESSO DE ETL: EXTRAÇÃO </font>
***

## Importação de todas as bibliotecas utilizadas no projeto

In [2]:
import pandas as pd
import numpy as np
from glob import glob
from unidecode import unidecode
from datetime import datetime
import pyodbc

pd.set_option('mode.chained_assignment', None)

***

## Agrupamento dos arquivos numa única lista
Devido às mudanças de <i>encoding</i> no decorrer dos anos, foi necessário fazer uma extração por partes.

In [3]:
arquivos = sorted(glob('pda-prouni-20*.csv'))

In [4]:
dados = []
for j in range(12):
    t = pd.read_csv(arquivos[j], sep=';', encoding='latin-1')
    dados.append(t)

In [5]:
for j in range(12,15):
    t = pd.read_csv(arquivos[j], sep=';', encoding='utf-8')
    dados.append(t)

In [6]:
dados.append(pd.read_csv(arquivos[15], sep=';', encoding='latin-1'))

In [7]:
dados[0].head(3)

,ANO_CONCESSAO_BOLSA,CODIGO_EMEC_IES_BOLSA,NOME_IES_BOLSA,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,NOME_CURSO_BOLSA,NOME_TURNO_CURSO_BOLSA,CPF_BENEFICIARIO_BOLSA,SEXO_BENEFICIARIO_BOLSA,RACA_BENEFICIARIO_BOLSA,DT_NASCIMENTO_BENEFICIARIO,BENEFICIARIO_DEFICIENTE_FISICO,REGIAO_BENEFICIARIO_BOLSA,SIGLA_UF_BENEFICIARIO_BOLSA,MUNICIPIO_BENEFICIARIO_BOLSA
0,2005,423,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Enfermagem,Integral,***264740**,Feminino,Branca,17-02-1987,NÃO,SUL,RS,SANTO ÂNGELO
1,2005,423,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Serviço Social,Noturno,***976940**,Feminino,Parda,14-06-1986,NÃO,SUL,RS,FREDERICO WESTPHALEN
2,2005,423,UNIVERSIDADE REGIONAL INTEGRADA DO ALTO URUGUA...,BOLSA PARCIAL 50%,PRESENCIAL,Serviço Social,Noturno,***045340**,Feminino,Parda,06-03-1984,NÃO,SUL,RS,FREDERICO WESTPHALEN


***

# <font color=#257C53 size=5> 4 PROCESSO DE ETL: TRANSFORMAÇÃO </font>
***

## Construção da Tabela Raça

|Colunas|Tipo|
|---|---|
|COD_RACA|INT|
|DESC_RACA|NVARCHAR|

In [8]:
raca = []
for i in range(len(dados)-1):
    t = dados[i].RACA_BENEFICIARIO_BOLSA
    raca.extend(t)

In [9]:
racas = np.unique(raca)

In [10]:
TAB_RACA = pd.DataFrame(data=racas[:6], columns=['DESC_RACA'])
TAB_RACA.index.names = ['COD_RACA']
TAB_RACA

,DESC_RACA
COD_RACA,
0,Amarela
1,Branca
2,Indígena
3,Não Informada
4,Parda
5,Preta


***

## Construção da Tabela Local

|Colunas|Tipo|
|---|---|
|COD_MUNICIPIO|INT|
|REGIAO|NVARCHAR|
|SIGLA_UF|NVARCHAR|
|MUNICIPIO|NVARCHAR|

In [11]:
TAB_LOCAL = pd.concat(dados[i][['REGIAO_BENEFICIARIO_BOLSA', 
                    'SIGLA_UF_BENEFICIARIO_BOLSA', 
                    'MUNICIPIO_BENEFICIARIO_BOLSA']].drop_duplicates(
    keep='first',subset='MUNICIPIO_BENEFICIARIO_BOLSA') for i in range(len(dados)-1))

In [12]:
TAB_LOCAL.rename(columns= {'REGIAO_BENEFICIARIO_BOLSA':'REGIAO',
                          'SIGLA_UF_BENEFICIARIO_BOLSA':'SIGLA_UF',
                          'MUNICIPIO_BENEFICIARIO_BOLSA':'DESC_MUNICIPIO'}, inplace=True)

In [13]:
TAB_LOCAL = pd.concat([TAB_LOCAL, dados[15][['REGIAO_BENEFICIARIO', 
                                             'UF_BENEFICIARIO', 
                                             'MUNICIPIO_BENEFICIARIO']].drop_duplicates(keep='first', 
                                                                                        subset='MUNICIPIO_BENEFICIARIO').rename(
    columns={'REGIAO_BENEFICIARIO':'REGIAO', 'UF_BENEFICIARIO':'SIGLA_UF', 'MUNICIPIO_BENEFICIARIO':'DESC_MUNICIPIO'})])

In [14]:
TAB_LOCAL.DESC_MUNICIPIO = TAB_LOCAL.DESC_MUNICIPIO.astype('str')
TAB_LOCAL.REGIAO = TAB_LOCAL.REGIAO.str.title()
TAB_LOCAL.DESC_MUNICIPIO = TAB_LOCAL.DESC_MUNICIPIO.str.title()

In [15]:
mun_unidecode = []
for i in range(len(TAB_LOCAL)):
    t = unidecode(TAB_LOCAL.DESC_MUNICIPIO.iloc[i])
    mun_unidecode.append(t)

In [16]:
TAB_LOCAL.DESC_MUNICIPIO = mun_unidecode

In [17]:
TAB_LOCAL.drop_duplicates(subset='DESC_MUNICIPIO',inplace=True, keep='first')
TAB_LOCAL.DESC_MUNICIPIO.replace('Nan','', inplace=True)
TAB_LOCAL.dropna(how='any', inplace=True)
TAB_LOCAL.reset_index(drop=True, inplace=True)

In [18]:
#TAB_LOCAL = TAB_LOCAL.append({'DESC_MUNICIPIO':'Não Informado', 'SIGLA_UF':'Não Informado', 'REGIAO':'Não Informado'}, 
                             #ignore_index=True)

In [19]:
TAB_LOCAL.index.names = ['COD_MUNICIPIO']
TAB_LOCAL.head(3)

,REGIAO,SIGLA_UF,DESC_MUNICIPIO
COD_MUNICIPIO,,,
0,Sul,RS,Santo Angelo
1,Sul,RS,Frederico Westphalen
2,Sul,PR,Sao Jose Dos Pinhais


<font size=2> Obs.: municípios sem acentuação </font>


***

## Construção da Tabela IES

|Colunas|Tipo|
|---|---|
|COD_EMEC_IES_BOLSA|INT|
|NOME_IES_BOLSA|NVARCHAR|

In [20]:
TAB_IES = pd.concat((dados[i][['CODIGO_EMEC_IES_BOLSA','NOME_IES_BOLSA']]).drop_duplicates(
    keep='first' ,subset='CODIGO_EMEC_IES_BOLSA') for i in range(len(dados)))

In [21]:
TAB_IES.NOME_IES_BOLSA = TAB_IES.NOME_IES_BOLSA.str.title()

In [22]:
TAB_IES.drop_duplicates(keep='first' ,subset='CODIGO_EMEC_IES_BOLSA', inplace=True)
TAB_IES.dropna(how='all', inplace=True)
TAB_IES.CODIGO_EMEC_IES_BOLSA = pd.to_numeric(TAB_IES.CODIGO_EMEC_IES_BOLSA, downcast='integer')
TAB_IES.NOME_IES_BOLSA.fillna('Não Informado', inplace=True)

In [23]:
TAB_IES.set_index('CODIGO_EMEC_IES_BOLSA', inplace=True)

In [24]:
TAB_IES.sort_index(inplace=True)

In [25]:
TAB_IES.index.names = ['COD_EMEC_IES_BOLSA']
TAB_IES.head(3)

,NOME_IES_BOLSA
COD_EMEC_IES_BOLSA,
10,Pontifícia Universidade Católica Do Paraná
11,Universidade Católica De Pernambuco
13,Universidade De Caxias Do Sul


***

## Construção da Tabela Registro

|Colunas|Tipo|
|---|---|
|COD_ID|INT IDENTITY|
|ANO_CONCESSAO_BOLSA|INT|
|COD_RACA|INT|
|COD_MUNICIPIO|INT|
|COD_EMEC_IES_BOLSA|INT|
|CPF_BENEFICIARIO_BOLSA|NVARCHAR|
|SEXO_BENEFICIARIO_BOLSA|NVARCHAR|
|NOME_CURSO_BOLSA|NVARCHAR|
|DT_NASCIMENTO_BENEFICIARIO|DATE|
|BENEFICIARIO_DEFICIENTE_FISICO|INT|
|TIPO_BOLSA|NVARCHAR|
|MODALIDADE_ENSINO_BOLSA|NVARCHAR|

In [26]:
for j in range(len(dados)-1):
    dados[j].MUNICIPIO_BENEFICIARIO_BOLSA = dados[j].MUNICIPIO_BENEFICIARIO_BOLSA.astype(str)

In [27]:
dados[14].dropna(inplace=True)

In [28]:
def formatacao_tabela(i):
    
    #CHAMANDO TABELA
    TAB_REG = dados[i][['ANO_CONCESSAO_BOLSA', 'CODIGO_EMEC_IES_BOLSA', 'CPF_BENEFICIARIO_BOLSA', 'TIPO_BOLSA',
                        'SEXO_BENEFICIARIO_BOLSA','BENEFICIARIO_DEFICIENTE_FISICO', 'DT_NASCIMENTO_BENEFICIARIO',
                       'MODALIDADE_ENSINO_BOLSA', 'NOME_CURSO_BOLSA']]

    # GERANDO MAPAS IDENTIFICADORES
    mapa_raca = {'Amarela':0, 'Branca':1, 'Indígena':2, 'Não Informada':3, 'Parda':4, 'Preta':5}
    mapa_sexo = {'Feminino':'F', 'Masculino': 'M', 'F':'F', 'M':'M'}
    mapa_tipo =  {'BOLSA COMPLEMENTAR 25%':'25%', 'BOLSA INTEGRAL':'100%', 'BOLSA PARCIAL 50%':'50%',
           'INTEGRAL':'100%', 'PARCIAL':'50%'}
    mapa_pcd = {'N':0, 'NÃO':0, 'S':1, 'SIM':1}
    mapa_modalidade = {'EAD':'EaD', 'EDUCAÇÃO A DISTÂNCIA':'EaD', 'PRESENCIAL':'Presencial', 'Presencial':'Presencial'}

    mapa_local = TAB_LOCAL.DESC_MUNICIPIO.to_dict()
    mapa_local = {v: k for k, v in mapa_local.items()}

    #FORMATANDO TABELA MUNICIPIO
      
    mun_unidecode = []
    for j in range(len(dados[i])):
        t = unidecode(dados[i].MUNICIPIO_BENEFICIARIO_BOLSA.iloc[j])
        mun_unidecode.append(t.title())
        
    TAB_REG['COD_MUNICIPIO'] = mun_unidecode
    TAB_REG['COD_MUNICIPIO'] = TAB_REG['COD_MUNICIPIO'].map(mapa_local)
    #TAB_REG.fillna({'COD_MUNICIPIO':5360}, inplace=True)

    # ORGANIZANDO COLUNAS DA TABELA
    TAB_REG['COD_RACA'] = dados[i]['RACA_BENEFICIARIO_BOLSA'].map(mapa_raca, 
                                                                  na_action='ignore').convert_dtypes(convert_integer=True)
    TAB_REG['SEXO_BENEFICIARIO_BOLSA'] = dados[i].SEXO_BENEFICIARIO_BOLSA.map(mapa_sexo, na_action='ignore')
    TAB_REG['CPF_BENEFICIARIO_BOLSA'] = dados[i].CPF_BENEFICIARIO_BOLSA.str.strip('*')
    TAB_REG['TIPO_BOLSA'] = TAB_REG['TIPO_BOLSA'].map(mapa_tipo, na_action='ignore')
    TAB_REG['BENEFICIARIO_DEFICIENTE_FISICO']= TAB_REG.BENEFICIARIO_DEFICIENTE_FISICO.map(mapa_pcd, na_action='ignore')
    TAB_REG['MODALIDADE_ENSINO_BOLSA']= TAB_REG.MODALIDADE_ENSINO_BOLSA.map(mapa_modalidade, na_action='ignore')
    TAB_REG['NOME_CURSO_BOLSA'] = TAB_REG.NOME_CURSO_BOLSA.str.title()
    
    # REDEFININDO OS TIPOS DAS COLUNAS
    TAB_REG['ANO_CONCESSAO_BOLSA'] = TAB_REG['ANO_CONCESSAO_BOLSA'].astype('int64')
    TAB_REG['CODIGO_EMEC_IES_BOLSA'] = TAB_REG['CODIGO_EMEC_IES_BOLSA'].astype('int64')
    #TAB_REG['COD_MUNICIPIO'] = TAB_REG['COD_MUNICIPIO'].astype('int64')
    TAB_REG['CPF_BENEFICIARIO_BOLSA'] = TAB_REG['CPF_BENEFICIARIO_BOLSA'].astype(str)
    TAB_REG['SEXO_BENEFICIARIO_BOLSA'] = TAB_REG['SEXO_BENEFICIARIO_BOLSA'].astype(str)
    TAB_REG['NOME_CURSO_BOLSA'] = TAB_REG['NOME_CURSO_BOLSA'].astype(str)
    TAB_REG['TIPO_BOLSA'] = TAB_REG['TIPO_BOLSA'].astype(str)
    TAB_REG['MODALIDADE_ENSINO_BOLSA'] = TAB_REG['MODALIDADE_ENSINO_BOLSA'].astype(str)
    
    # ORDENANDO TABELA FINAL
    ordem = ['ANO_CONCESSAO_BOLSA', 'COD_RACA', 'COD_MUNICIPIO', 'CODIGO_EMEC_IES_BOLSA', 'CPF_BENEFICIARIO_BOLSA', 
            'SEXO_BENEFICIARIO_BOLSA', 'NOME_CURSO_BOLSA', 'DT_NASCIMENTO_BENEFICIARIO', 'BENEFICIARIO_DEFICIENTE_FISICO', 
            'TIPO_BOLSA', 'MODALIDADE_ENSINO_BOLSA']
    TAB_REG = TAB_REG[ordem]

    return TAB_REG

In [29]:
TAB = []
for j in range(len(dados)-1):
    t = formatacao_tabela(j)
    TAB.append(t)

<b>No ano de 2020 houve uma fuga no padrão de preenchimento das tabelas, portanto fez-se necessário realizar um processo de transformação independente.</b>

In [30]:
    #CHAMANDO TABELA
    TAB_REG15 = dados[15][['ANO_CONCESSAO_BOLSA', 'CODIGO_EMEC_IES_BOLSA', 'CPF_BENEFICIARIO', 'TIPO_BOLSA',
                        'SEXO_BENEFICIARIO','BENEFICIARIO_DEFICIENTE_FISICO', 'DATA_NASCIMENTO',
                       'MODALIDADE_ENSINO_BOLSA', 'NOME_CURSO_BOLSA']]

    # GERANDO MAPAS IDENTIFICADORES
    mapa_raca = {'Amarela':0, 'Branca':1, 'Indígena':2, 'Não Informada':3, 'Parda':4, 'Preta':5}
    mapa_tipo =  {'BOLSA COMPLEMENTAR 25%':'25%', 'BOLSA INTEGRAL':'100%', 'BOLSA PARCIAL 50%':'50%',
           'INTEGRAL':'100%', 'PARCIAL':'50%'}
    mapa_pcd = {'N':0, 'NÃO':0, 'S':1, 'SIM':1}
    mapa_modalidade = {'EAD':'EaD', 'EDUCAÇÃO A DISTÂNCIA':'EaD', 'PRESENCIAL':'Presencial', 'Presencial':'Presencial'}

    mapa_local = TAB_LOCAL.DESC_MUNICIPIO.to_dict()
    mapa_local = {v: k for k, v in mapa_local.items()}

    
    #FORMATANDO TABELA MUNICIPIO
    mun_unidecode = []
    for i in range(len(dados[15])):
        t = unidecode(dados[15].MUNICIPIO_BENEFICIARIO.iloc[i].title())
        mun_unidecode.append(t)
        
    TAB_REG15['COD_MUNICIPIO'] = mun_unidecode
    TAB_REG15['COD_MUNICIPIO'] = TAB_REG15['COD_MUNICIPIO'].map(mapa_local)
    #TAB_REG15.fillna({'COD_MUNICIPIO':5360}, inplace=True)
    #TAB_REG15['COD_MUNICIPIO'] = TAB_REG15['COD_MUNICIPIO'].astype('int64')
    
    # FORMATANDO TABELA CPF
    
    cpf = []
    for i in range(len(dados[15])):
        t = dados[15].CPF_BENEFICIARIO[i][:3] + dados[15].CPF_BENEFICIARIO[i][-2:] 
        cpf.append(t)
    TAB_REG15['CPF_BENEFICIARIO_BOLSA'] = cpf
    
    # ORGANIZANDO COLUNAS DA TABELA
    TAB_REG15['COD_RACA'] = dados[15]['RACA_BENEFICIARIO'].map(mapa_raca, 
                                                                  na_action='ignore').convert_dtypes(convert_integer=True)
    TAB_REG15['SEXO_BENEFICIARIO_BOLSA'] = dados[15].SEXO_BENEFICIARIO    
    TAB_REG15['TIPO_BOLSA'] = TAB_REG15['TIPO_BOLSA'].map(mapa_tipo, na_action='ignore')
    TAB_REG15['BENEFICIARIO_DEFICIENTE_FISICO']= TAB_REG15.BENEFICIARIO_DEFICIENTE_FISICO.map(mapa_pcd, na_action='ignore')
    TAB_REG15['MODALIDADE_ENSINO_BOLSA']= TAB_REG15.MODALIDADE_ENSINO_BOLSA.map(mapa_modalidade, na_action='ignore')
    TAB_REG15['NOME_CURSO_BOLSA'] = TAB_REG15.NOME_CURSO_BOLSA.str.title()
    TAB_REG15['NOME_CURSO_BOLSA'] = TAB_REG15['NOME_CURSO_BOLSA'].fillna('Não Informado')
    TAB_REG15['DT_NASCIMENTO_BENEFICIARIO'] = TAB_REG15['DATA_NASCIMENTO']
    TAB_REG15['ANO_CONCESSAO_BOLSA'] = TAB_REG15['ANO_CONCESSAO_BOLSA'].astype('int64')
    TAB_REG15['CODIGO_EMEC_IES_BOLSA'] = TAB_REG15['CODIGO_EMEC_IES_BOLSA'].astype('int64')

    
    ordem = ['ANO_CONCESSAO_BOLSA', 'COD_RACA', 'COD_MUNICIPIO', 'CODIGO_EMEC_IES_BOLSA', 'CPF_BENEFICIARIO_BOLSA', 
            'SEXO_BENEFICIARIO_BOLSA', 'NOME_CURSO_BOLSA', 'DT_NASCIMENTO_BENEFICIARIO', 'BENEFICIARIO_DEFICIENTE_FISICO', 
            'TIPO_BOLSA', 'MODALIDADE_ENSINO_BOLSA']
    TAB_REG15 = TAB_REG15[ordem]


In [31]:
TAB.append(TAB_REG15)

In [32]:
for i in range(len(TAB)):
    TAB[i].rename(columns={'CODIGO_EMEC_IES_BOLSA':'COD_EMEC_IES_BOLSA'}, inplace=True)

In [33]:
for i in range(len(TAB)):
    if TAB[i].DT_NASCIMENTO_BENEFICIARIO[0][2] == '/':
        TAB[i].DT_NASCIMENTO_BENEFICIARIO = TAB[i].DT_NASCIMENTO_BENEFICIARIO.apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
    elif TAB[i].DT_NASCIMENTO_BENEFICIARIO[0][2] == '-':
        TAB[i].DT_NASCIMENTO_BENEFICIARIO = TAB[i].DT_NASCIMENTO_BENEFICIARIO.apply(lambda x: datetime.strptime(x, '%d-%m-%Y'))

### Construindo coluna de Idades

In [34]:
for i in range(len(TAB)):
    TAB[i]['IDADE'] = TAB[i].ANO_CONCESSAO_BOLSA - TAB[i].DT_NASCIMENTO_BENEFICIARIO.apply(lambda x: str(x)[:4]).astype(int)

Retirando possíveis falhas de preenchimento com pouca representatividade amostral e outliers

In [35]:
for i in range(len(TAB)):
    for j in range(len(TAB[i])):
        if TAB[i].IDADE[j] < 15:
            TAB[i].IDADE[j] = None
        elif TAB[i].IDADE[j] > 115:
            TAB[i].IDADE[j] = None

In [36]:
TAB[0].head(3)

,ANO_CONCESSAO_BOLSA,COD_RACA,COD_MUNICIPIO,COD_EMEC_IES_BOLSA,CPF_BENEFICIARIO_BOLSA,SEXO_BENEFICIARIO_BOLSA,NOME_CURSO_BOLSA,DT_NASCIMENTO_BENEFICIARIO,BENEFICIARIO_DEFICIENTE_FISICO,TIPO_BOLSA,MODALIDADE_ENSINO_BOLSA,IDADE
0,2005,1,0.0,423,264740,F,Enfermagem,1987-02-17 00:00:00,0,50%,Presencial,18.0
1,2005,4,1.0,423,976940,F,Serviço Social,1986-06-14 00:00:00,0,50%,Presencial,19.0
2,2005,4,1.0,423,045340,F,Serviço Social,1984-03-06 00:00:00,0,50%,Presencial,21.0


***

***

# <font color=#257C53 size=5> 5 PROCESSO DE ETL: CARGA </font>
***

## Conexão com o SQL Server

In [36]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Integer
from sqlalchemy.orm import sessionmaker

In [37]:
server = 'DESKTOP-VASG2SS'
database = 'DADOS_PROUNI'
driver = 'SQL Server Native Client 11.0'
username = ''
password = ''
database_connection = f'mssql://{username}:{password}@{server}/{database}?driver={driver}'

In [38]:
engine = create_engine(database_connection)

In [39]:
conn = engine.connect()

***

## Carga das tabelas

In [40]:
TAB_RACA.to_sql('TAB_RACA', engine, if_exists='append')

In [41]:
TAB_LOCAL.to_sql('TAB_LOCAL', engine, if_exists='append')

In [42]:
TAB_IES.to_sql('TAB_IES', engine, if_exists='append')

In [43]:
for i in range(len(TAB)):
    TAB[i].to_sql('TAB_REGISTRO', engine, if_exists='append', index=False)
    print(f'Carga da tabela {TAB[i].iloc[1][0]} finalizada.')

Carga da tabela 2005 finalizada.
Carga da tabela 2006 finalizada.
Carga da tabela 2007 finalizada.
Carga da tabela 2008 finalizada.
Carga da tabela 2009 finalizada.
Carga da tabela 2010 finalizada.
Carga da tabela 2011 finalizada.
Carga da tabela 2012 finalizada.
Carga da tabela 2013 finalizada.
Carga da tabela 2014 finalizada.
Carga da tabela 2015 finalizada.
Carga da tabela 2016 finalizada.
Carga da tabela 2017 finalizada.
Carga da tabela 2018 finalizada.
Carga da tabela 2019 finalizada.
Carga da tabela 2020 finalizada.


In [44]:
conn.close()